# AI - Vacuum agent, empty program

Created (R. Basili, Nov. 2020)  
Modified (C.D. Hromei, Oct 2024)

## Exercises

1. Fill the program of the `Agent` in order to reproduce the behaviour of random moves.
2. Fill the program of the `Agent` in order to search for the closest dirty room, go there and `Suck`.  
    **Hint**: you should modify the `Perception` method of the Environment as well to make it Completely Observable.
3. Modify the `Environment` locations into a 2x2 matrix instead of a row.
4. Modify the `Execute action` method of the Environment in order to implement a probabilistic failure of the action chosen by the Agent. In that case, the action should fail and nothing happens!

In [ ]:
! pip install Pillow numpy tk

In [17]:
from tkinter import *
import random
from agents import *

loc_A, loc_B, loc_C = (0, 0), (1, 0), (2,0)  # The three locations for the Vacuum world

MyImage = None

class Gui(Environment):

    """This GUI environment has three locations, A, B and C. Each can be Dirty
    or Clean. The agent perceives its location and the location's
    status."""

    def __init__(self, root, height=400, width=520):
        super().__init__()
        self.status = {loc_A: 'Dirty',
                       loc_B: 'Dirty', 
                       loc_C: 'Clean'}
        self.root = root
        self.height = height
        self.width = width
        self.canvas = None
        self.buttons = []
        self.create_canvas()
        self.create_buttons()

    def thing_classes(self):
        """The list of things which can be used in the environment."""
        return [Wall, Dirt, ReflexVacuumAgent, RandomVacuumAgent,
                TableDrivenVacuumAgent, ModelBasedVacuumAgent]

    def percept(self, agent):
        """Returns the agent's location, and the location status (Dirty/Clean)."""
        print('Perceived Location=', agent.location,' - Perceived StatusAtLoc=',  self.status[agent.location])
        return (agent.location, self.status[agent.location]) #(agent.location, self.status[agent.location])

    def execute_action(self, agent, action):
        """Change the location status (Dirty/Clean); track performance.
        Score 10 for each dirt cleaned; -1 for each move."""
        print('Executing action: ', action)
        if action == 'Right':
            if agent.location == loc_A:
                agent.location=loc_B
            else:
                agent.location=loc_C
            agent.performance -= 1
        elif action == 'Left':
            if agent.location == loc_C:
                agent.location = loc_B
            else:
                agent.location=loc_A
            agent.performance -= 1
        elif action == 'Suck':
            if self.status[agent.location] == 'Dirty':
                if agent.location == loc_A:
                    self.buttons[0].config(bg='white', activebackground='light grey')
                elif agent.location == loc_B:
                    self.buttons[1].config(bg='white', activebackground='light grey')
                else:
                    self.buttons[2].config(bg='white', activebackground='light grey')
                agent.performance += 10
            self.status[agent.location] = 'Clean'

    def default_location(self, thing):
        """Agents start in either location at random."""
        return random.choice([loc_A, loc_B, loc_C])

    ###################################################################################################
    #                        INTERFACE METHODS NOT USEFULL FOR THE AGENT/ENVIRONMENT
    ###################################################################################################
    def create_canvas(self):
        """Creates Canvas element in the GUI."""
        self.canvas = Canvas(
            self.root,
            width=self.width,
            height=self.height,
            background='powder blue')
        self.canvas.pack(side='bottom')

    def create_buttons(self):
        """Creates the buttons required in the GUI."""
        button_left = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_left.config(command=lambda btn=button_left: self.dirt_switch(btn))
        self.buttons.append(button_left)
        button_left_window = self.canvas.create_window(130, 200, anchor=N, window=button_left)

        button_middle = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_middle.config(command=lambda btn=button_middle: self.dirt_switch(btn))
        self.buttons.append(button_middle)
        button_middle_window = self.canvas.create_window(250, 200, anchor=N, window=button_middle)

        button_right = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_right.config(command=lambda btn=button_right: self.dirt_switch(btn))
        self.buttons.append(button_right)
        button_right_window = self.canvas.create_window(370, 200, anchor=N, window=button_right)

    def dirt_switch(self, button):
        """Gives user the option to put dirt in any tile."""
        bg_color = button['bg']
        if bg_color == 'saddle brown':
            button.config(bg='white', activebackground='light grey')
        elif bg_color == 'white':
            button.config(bg='saddle brown', activebackground='light goldenrod')

    def read_env(self):
        """Reads the current state of the GUI."""
        for i, btn in enumerate(self.buttons):
            if i == 0:
                if btn['bg'] == 'white':
                    self.status[loc_A] = 'Clean'
                else:
                    self.status[loc_A] = 'Dirty'
            elif i == 1:
                if btn['bg'] == 'white':
                    self.status[loc_B] = 'Clean'
                else:
                    self.status[loc_B] = 'Dirty'
            else:
                if btn['bg'] == 'white':
                    self.status[loc_C] = 'Clean'
                else:
                    self.status[loc_C] = 'Dirty'

    def update_env(self, agent):
        """Updates the GUI according to the agent's action."""
        self.read_env()
        # print(self.status)
        before_step = agent.location
        self.step()   #execute_actions
        print('Current Environment Status: ', self.status)
        print('Current Agent Location: ',agent.location)
        move_agent(self, agent, before_step)

###################################################################################################
#                                           THE AGENT
###################################################################################################

def ReflexVacuumAgent():
    """A reflex agent for the two-state vacuum environment. [Figure 2.8]
    >>> agent = ReflexVacuumAgent()
    >>> environment = TrivialVacuumEnvironment()
    >>> environment.add_thing(agent)
    >>> environment.run()
    >>> environment.status == {(1,0):'Clean' , (0,0) : 'Clean'}
    True
    """
    
    def program(percept):
        
        location, status = percept
        print('Perceived Status: ', percept)
        
        # TODO: fill this method and remove the next print!
        print('My program is empty, fill me up!')
        return "Stay"

    return Agent(program)

###################################################################################################
#                            INTERFACE METHODS TO CREATE AND MOVE THE AGENT
###################################################################################################

def create_agent(env, agent):
    """Creates the agent in the GUI and is kept independent of the environment."""
    env.add_thing(agent)

    if agent.location == (0, 0):
        env.text = env.canvas.create_text(120, 140, font="Helvetica 10 bold italic", text="Agent")
        env.agent_logo = env.canvas.create_image(80, 100, image=MyImage)
        
    elif agent.location == (2, 0):
        env.text = env.canvas.create_text(360, 140, font="Helvetica 10 bold italic", text="Agent")
        env.agent_logo = env.canvas.create_image(320, 100, image=MyImage)
    elif agent.location == (1, 0):
        env.text = env.canvas.create_text(240, 140, font="Helvetica 10 bold italic", text="Agent")
        env.agent_logo = env.canvas.create_image(200, 100, image=MyImage)

def move_agent(env, agent, before_step):
    """Moves the agent in the GUI when 'next' button is pressed."""
    if agent.location == before_step:
        pass
    else:
        if agent.location == (2, 0) and before_step == (1,0):
            env.canvas.move(env.text, 120, 0)
            env.canvas.move(env.agent_logo, 120, 0)
        elif agent.location == (1, 0) and  before_step == (2,0): 
            env.canvas.move(env.text, -120, 0)
            env.canvas.move(env.agent_logo, -120, 0)
        elif agent.location == (1, 0) and  before_step == (0,0): 
            env.canvas.move(env.text, 120, 0)
            env.canvas.move(env.agent_logo, 120, 0)
        elif agent.location == (0, 0) and before_step == (1,0):
            env.canvas.move(env.text, -120, 0)
            env.canvas.move(env.agent_logo, -120, 0)

# TODO: Add more agents to the environment.
# TODO: Expand the environment to XYEnvironment.

###################################################################################################
#                                           MAIN LOOP
###################################################################################################
def main():
    """The main function of the program."""
    root = Tk()
    root.title("Vacuum Environment")
    root.geometry("500x500")
    root.resizable(0, 0)
    frame = Frame(root, bg='black')
    
    next_button = Button(frame, text='Next', height=2, width=6, padx=2, pady=2)
    next_button.pack(side='left')
    frame.pack(side='bottom')
    
    env = Gui(root)
    agent = ReflexVacuumAgent()
    create_agent(env, agent)
    
    next_button.config(command=lambda: env.update_env(agent))
    
    root.mainloop()


if __name__ == "__main__":
    main()


**Esercizion 1**
Fill the program of the `Agent` in order to reproduce the behaviour of random moves.

**Soluzione :** 
Ho utilizzato la variabile `status` per verificare se la stanza fosse sporca o pulita. Nel caso la stanza sia sporca, l'agente procede con la pulizia; altrimenti, sceglie casualmente un movimento tra `Right` e `Left`.



In [18]:
from tkinter import *
import random
from agents import *

loc_A, loc_B, loc_C = (0, 0), (1, 0), (2,0)  # The three locations for the Vacuum world

MyImage = None

class Gui(Environment):

    """This GUI environment has three locations, A, B and C. Each can be Dirty
    or Clean. The agent perceives its location and the location's
    status."""

    def __init__(self, root, height=400, width=520):
        super().__init__()
        self.status = {loc_A: 'Dirty',
                       loc_B: 'Dirty', 
                       loc_C: 'Clean'}
        self.root = root
        self.height = height
        self.width = width
        self.canvas = None
        self.buttons = []
        self.create_canvas()
        self.create_buttons()

    def thing_classes(self):
        """The list of things which can be used in the environment."""
        return [Wall, Dirt, ReflexVacuumAgent, RandomVacuumAgent,
                TableDrivenVacuumAgent, ModelBasedVacuumAgent]

    def percept(self, agent):
        """Returns the agent's location, and the location status (Dirty/Clean)."""
        print('Perceived Location=', agent.location,' - Perceived StatusAtLoc=',  self.status[agent.location])
        return (agent.location, self.status[agent.location]) #(agent.location, self.status[agent.location])

    def execute_action(self, agent, action):
        """Change the location status (Dirty/Clean); track performance.
        Score 10 for each dirt cleaned; -1 for each move."""
        print('Executing action: ', action)
        if action == 'Right':
            if agent.location == loc_A:
                agent.location=loc_B
            else:
                agent.location=loc_C
            agent.performance -= 1
        elif action == 'Left':
            if agent.location == loc_C:
                agent.location = loc_B
            else:
                agent.location=loc_A
            agent.performance -= 1
        elif action == 'Suck':
            if self.status[agent.location] == 'Dirty':
                if agent.location == loc_A:
                    self.buttons[0].config(bg='white', activebackground='light grey')
                elif agent.location == loc_B:
                    self.buttons[1].config(bg='white', activebackground='light grey')
                else:
                    self.buttons[2].config(bg='white', activebackground='light grey')
                agent.performance += 10
            self.status[agent.location] = 'Clean'

    def default_location(self, thing):
        """Agents start in either location at random."""
        return random.choice([loc_A, loc_B, loc_C])

    ###################################################################################################
    #                        INTERFACE METHODS NOT USEFULL FOR THE AGENT/ENVIRONMENT
    ###################################################################################################
    def create_canvas(self):
        """Creates Canvas element in the GUI."""
        self.canvas = Canvas(
            self.root,
            width=self.width,
            height=self.height,
            background='powder blue')
        self.canvas.pack(side='bottom')

    def create_buttons(self):
        """Creates the buttons required in the GUI."""
        button_left = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_left.config(command=lambda btn=button_left: self.dirt_switch(btn))
        self.buttons.append(button_left)
        button_left_window = self.canvas.create_window(130, 200, anchor=N, window=button_left)

        button_middle = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_middle.config(command=lambda btn=button_middle: self.dirt_switch(btn))
        self.buttons.append(button_middle)
        button_middle_window = self.canvas.create_window(250, 200, anchor=N, window=button_middle)

        button_right = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_right.config(command=lambda btn=button_right: self.dirt_switch(btn))
        self.buttons.append(button_right)
        button_right_window = self.canvas.create_window(370, 200, anchor=N, window=button_right)

    def dirt_switch(self, button):
        """Gives user the option to put dirt in any tile."""
        bg_color = button['bg']
        if bg_color == 'saddle brown':
            button.config(bg='white', activebackground='light grey')
        elif bg_color == 'white':
            button.config(bg='saddle brown', activebackground='light goldenrod')

    def read_env(self):
        """Reads the current state of the GUI."""
        for i, btn in enumerate(self.buttons):
            if i == 0:
                if btn['bg'] == 'white':
                    self.status[loc_A] = 'Clean'
                else:
                    self.status[loc_A] = 'Dirty'
            elif i == 1:
                if btn['bg'] == 'white':
                    self.status[loc_B] = 'Clean'
                else:
                    self.status[loc_B] = 'Dirty'
            else:
                if btn['bg'] == 'white':
                    self.status[loc_C] = 'Clean'
                else:
                    self.status[loc_C] = 'Dirty'

    def update_env(self, agent):
        """Updates the GUI according to the agent's action."""
        self.read_env()
        # print(self.status)
        before_step = agent.location
        self.step()   #execute_actions
        print('Current Environment Status: ', self.status)
        print('Current Agent Location: ',agent.location)
        move_agent(self, agent, before_step)

###################################################################################################
#                                           THE AGENT
###################################################################################################

def ReflexVacuumAgent():
    """A reflex agent for the two-state vacuum environment. [Figure 2.8]
    >>> agent = ReflexVacuumAgent()
    >>> environment = TrivialVacuumEnvironment()
    >>> environment.add_thing(agent)
    >>> environment.run()
    >>> environment.status == {(1,0):'Clean' , (0,0) : 'Clean'}
    True
    """
    
    def program(percept):
        
        location, status = percept
        print('Perceived Status: ', percept)
        
        # TODO: fill this method and remove the next print!
        if status == 'Dirty':
            return 'Suck'
        if status == 'Clean':
            return random.choice(['Right','Left'])

        return "Stay"

    return Agent(program)

###################################################################################################
#                            INTERFACE METHODS TO CREATE AND MOVE THE AGENT
###################################################################################################

def create_agent(env, agent):
    """Creates the agent in the GUI and is kept independent of the environment."""
    env.add_thing(agent)

    if agent.location == (0, 0):
        env.text = env.canvas.create_text(120, 140, font="Helvetica 10 bold italic", text="Agent")
        env.agent_logo = env.canvas.create_image(80, 100, image=MyImage)
        
    elif agent.location == (2, 0):
        env.text = env.canvas.create_text(360, 140, font="Helvetica 10 bold italic", text="Agent")
        env.agent_logo = env.canvas.create_image(320, 100, image=MyImage)
    elif agent.location == (1, 0):
        env.text = env.canvas.create_text(240, 140, font="Helvetica 10 bold italic", text="Agent")
        env.agent_logo = env.canvas.create_image(200, 100, image=MyImage)

def move_agent(env, agent, before_step):
    """Moves the agent in the GUI when 'next' button is pressed."""
    if agent.location == before_step:
        pass
    else:
        if agent.location == (2, 0) and before_step == (1,0):
            env.canvas.move(env.text, 120, 0)
            env.canvas.move(env.agent_logo, 120, 0)
        elif agent.location == (1, 0) and  before_step == (2,0): 
            env.canvas.move(env.text, -120, 0)
            env.canvas.move(env.agent_logo, -120, 0)
        elif agent.location == (1, 0) and  before_step == (0,0): 
            env.canvas.move(env.text, 120, 0)
            env.canvas.move(env.agent_logo, 120, 0)
        elif agent.location == (0, 0) and before_step == (1,0):
            env.canvas.move(env.text, -120, 0)
            env.canvas.move(env.agent_logo, -120, 0)

# TODO: Add more agents to the environment.
# TODO: Expand the environment to XYEnvironment.

###################################################################################################
#                                           MAIN LOOP
###################################################################################################
def main():
    """The main function of the program."""
    root = Tk()
    root.title("Vacuum Environment")
    root.geometry("500x500")
    root.resizable(0, 0)
    frame = Frame(root, bg='black')
    
    next_button = Button(frame, text='Next', height=2, width=6, padx=2, pady=2)
    next_button.pack(side='left')
    frame.pack(side='bottom')
    
    env = Gui(root)
    agent = ReflexVacuumAgent()
    create_agent(env, agent)
    
    next_button.config(command=lambda: env.update_env(agent))
    
    root.mainloop()


if __name__ == "__main__":
    main()


**Esercizion 2**
Fill the program of the `Agent` in order to search for the closest dirty room, go there and `Suck`.  
        **Hint**: you should modify the `Perception` method of the Environment as well to make it Completely Observable.

**Soluzione**
Innanzitutto, ho modificato la funzione `percept` per restituire lo stato complessivo delle stanze. Successivamente, ho aggiornato la funzione `program`, sfruttando la variabile `status` per determinare se la stanza attuale fosse sporca o pulita. Le azioni dell’agente seguono queste logiche:

- Se la stanza è sporca, l'agente procede con la pulizia.
- Se la stanza è pulita, l'agente agisce come segue:
   - Se si trova in `loc_A`, l’unica azione possibile è spostarsi a destra.
   - Se si trova in `loc_C`, l’unica azione possibile è spostarsi a sinistra.
   - Se si trova in `loc_B`, l'agente sceglie se spostarsi a sinistra o a destra, a seconda di quale delle due stanze adiacenti sia sporca. Nel caso entrambe le stanze siano sporche, si dà priorità allo spostamento verso `loc_A`. 

Questa soluzione garantisce che l'agente agisca in modo efficiente per mantenere pulito l’ambiente in cui opera.


In [19]:
from tkinter import *
import random
from agents import *

loc_A, loc_B, loc_C = (0, 0), (1, 0), (2,0)  # The three locations for the Vacuum world

MyImage = None

class Gui(Environment):

    """This GUI environment has three locations, A, B and C. Each can be Dirty
    or Clean. The agent perceives its location and the location's
    status."""

    def __init__(self, root, height=400, width=520):
        super().__init__()
        self.status = {loc_A: 'Dirty',
                       loc_B: 'Dirty', 
                       loc_C: 'Clean'}
        self.root = root
        self.height = height
        self.width = width
        self.canvas = None
        self.buttons = []
        self.create_canvas()
        self.create_buttons()

    def thing_classes(self):
        """The list of things which can be used in the environment."""
        return [Wall, Dirt, ReflexVacuumAgent, RandomVacuumAgent,
                TableDrivenVacuumAgent, ModelBasedVacuumAgent]

    def percept(self, agent):
        """Returns the agent's location, and the location status (Dirty/Clean)."""
        print('Perceived Location=', agent.location,' - Perceived Status=',  self.status)
        return (agent.location, self.status) 

    def execute_action(self, agent, action):
        """Change the location status (Dirty/Clean); track performance.
        Score 10 for each dirt cleaned; -1 for each move."""
        print('Executing action: ', action)
        if action == 'Right':
            if agent.location == loc_A:
                agent.location=loc_B
            else:
                agent.location=loc_C
            agent.performance -= 1
        elif action == 'Left':
            if agent.location == loc_C:
                agent.location = loc_B
            else:
                agent.location=loc_A
            agent.performance -= 1
        elif action == 'Suck':
            if self.status[agent.location] == 'Dirty':
                if agent.location == loc_A:
                    self.buttons[0].config(bg='white', activebackground='light grey')
                elif agent.location == loc_B:
                    self.buttons[1].config(bg='white', activebackground='light grey')
                else:
                    self.buttons[2].config(bg='white', activebackground='light grey')
                agent.performance += 10
            self.status[agent.location] = 'Clean'

    def default_location(self, thing):
        """Agents start in either location at random."""
        return random.choice([loc_A, loc_B, loc_C])

    ###################################################################################################
    #                        INTERFACE METHODS NOT USEFULL FOR THE AGENT/ENVIRONMENT
    ###################################################################################################
    def create_canvas(self):
        """Creates Canvas element in the GUI."""
        self.canvas = Canvas(
            self.root,
            width=self.width,
            height=self.height,
            background='powder blue')
        self.canvas.pack(side='bottom')

    def create_buttons(self):
        """Creates the buttons required in the GUI."""
        button_left = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_left.config(command=lambda btn=button_left: self.dirt_switch(btn))
        self.buttons.append(button_left)
        button_left_window = self.canvas.create_window(130, 200, anchor=N, window=button_left)

        button_middle = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_middle.config(command=lambda btn=button_middle: self.dirt_switch(btn))
        self.buttons.append(button_middle)
        button_middle_window = self.canvas.create_window(250, 200, anchor=N, window=button_middle)

        button_right = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_right.config(command=lambda btn=button_right: self.dirt_switch(btn))
        self.buttons.append(button_right)
        button_right_window = self.canvas.create_window(370, 200, anchor=N, window=button_right)

    def dirt_switch(self, button):
        """Gives user the option to put dirt in any tile."""
        bg_color = button['bg']
        if bg_color == 'saddle brown':
            button.config(bg='white', activebackground='light grey')
        elif bg_color == 'white':
            button.config(bg='saddle brown', activebackground='light goldenrod')

    def read_env(self):
        """Reads the current state of the GUI."""
        for i, btn in enumerate(self.buttons):
            if i == 0:
                if btn['bg'] == 'white':
                    self.status[loc_A] = 'Clean'
                else:
                    self.status[loc_A] = 'Dirty'
            elif i == 1:
                if btn['bg'] == 'white':
                    self.status[loc_B] = 'Clean'
                else:
                    self.status[loc_B] = 'Dirty'
            else:
                if btn['bg'] == 'white':
                    self.status[loc_C] = 'Clean'
                else:
                    self.status[loc_C] = 'Dirty'

    def update_env(self, agent):
        """Updates the GUI according to the agent's action."""
        self.read_env()
        # print(self.status)
        before_step = agent.location
        self.step()   #execute_actions
        print('Current Environment Status: ', self.status)
        print('Current Agent Location: ',agent.location)
        move_agent(self, agent, before_step)

###################################################################################################
#                                           THE AGENT
###################################################################################################

def ReflexVacuumAgent():
    """A reflex agent for the two-state vacuum environment. [Figure 2.8]
    >>> agent = ReflexVacuumAgent()
    >>> environment = TrivialVacuumEnvironment()
    >>> environment.add_thing(agent)
    >>> environment.run()
    >>> environment.status == {(1,0):'Clean' , (0,0) : 'Clean'}
    True
    """
    
    def program(percept):
        
        location, status = percept
        print('Perceived Status: ', percept)
        
        # TODO: fill this method and remove the next print!
        if status[location] == 'Dirty':
            return 'Suck'
        if status[location] == 'Clean':
            if location  == loc_A:
                return 'Right'
            elif location == loc_B:
                if status[loc_A] == 'Dirty':
                    return 'Left'
                elif status[loc_C] == 'Dirty':
                    return 'Right'
                else:
                    return random.choice(['Left','Right'])
            elif location == loc_C:
                return 'Left'
            

        return "Stay"

    return Agent(program)

###################################################################################################
#                            INTERFACE METHODS TO CREATE AND MOVE THE AGENT
###################################################################################################

def create_agent(env, agent):
    """Creates the agent in the GUI and is kept independent of the environment."""
    env.add_thing(agent)

    if agent.location == (0, 0):
        env.text = env.canvas.create_text(120, 140, font="Helvetica 10 bold italic", text="Agent")
        env.agent_logo = env.canvas.create_image(80, 100, image=MyImage)
        
    elif agent.location == (2, 0):
        env.text = env.canvas.create_text(360, 140, font="Helvetica 10 bold italic", text="Agent")
        env.agent_logo = env.canvas.create_image(320, 100, image=MyImage)
    elif agent.location == (1, 0):
        env.text = env.canvas.create_text(240, 140, font="Helvetica 10 bold italic", text="Agent")
        env.agent_logo = env.canvas.create_image(200, 100, image=MyImage)

def move_agent(env, agent, before_step):
    """Moves the agent in the GUI when 'next' button is pressed."""
    if agent.location == before_step:
        pass
    else:
        if agent.location == (2, 0) and before_step == (1,0):
            env.canvas.move(env.text, 120, 0)
            env.canvas.move(env.agent_logo, 120, 0)
        elif agent.location == (1, 0) and  before_step == (2,0): 
            env.canvas.move(env.text, -120, 0)
            env.canvas.move(env.agent_logo, -120, 0)
        elif agent.location == (1, 0) and  before_step == (0,0): 
            env.canvas.move(env.text, 120, 0)
            env.canvas.move(env.agent_logo, 120, 0)
        elif agent.location == (0, 0) and before_step == (1,0):
            env.canvas.move(env.text, -120, 0)
            env.canvas.move(env.agent_logo, -120, 0)

# TODO: Add more agents to the environment.
# TODO: Expand the environment to XYEnvironment.

###################################################################################################
#                                           MAIN LOOP
###################################################################################################
def main():
    """The main function of the program."""
    root = Tk()
    root.title("Vacuum Environment")
    root.geometry("500x500")
    root.resizable(0, 0)
    frame = Frame(root, bg='black')
    
    next_button = Button(frame, text='Next', height=2, width=6, padx=2, pady=2)
    next_button.pack(side='left')
    frame.pack(side='bottom')
    
    env = Gui(root)
    agent = ReflexVacuumAgent()
    create_agent(env, agent)
    
    next_button.config(command=lambda: env.update_env(agent))
    
    root.mainloop()


if __name__ == "__main__":
    main()


**Esercizion 3**
Modify the `Environment` locations into a 2x2 matrix instead of a row.

**Soluzione :**

- Ho aggiunta una nuova stanza, `loc_D`, per ampliare l’ambiente dell’agente. Ora l'ambiente comprende quattro stanze: `loc_A`, `loc_B`, `loc_C` e `loc_D`.

- La funzione `execute_action` è stata modificata per includere le possibili transizioni dell’agente tra le quattro stanze. Questo  aggiornamento consente all’agente di spostarsi correttamente nell’ambiente esteso.

- La funzione `create_buttons` è stata adattata per includere un quarto pulsante nella GUI, rappresentante la stanza `loc_D`. Questo pulsante consente all'utente di interagire anche con la nuova posizione.

- La funzione `program` è stata aggiornata per consentire all’agente di selezionare in modo ottimale la stanza sporca più vicina. L’agente ora valuta la posizione delle stanze sporche e sceglie il percorso più efficace per la pulizia.

- Infine, le funzioni `create_agent` e `move_agent` sono state adattate per permettere alla GUI di visualizzare correttamente il movimento dell’agente nell’ambiente esteso. Questo permette di seguire visivamente gli spostamenti dell’agente tra tutte le stanze.


In [ ]:
from tkinter import *
import random
from agents import *

loc_A, loc_B, loc_C, loc_D= (0, 0), (1, 0), (0,1), (1,1) 

class Gui(Environment):

    """This GUI environment has three locations, A, B and C. Each can be Dirty
    or Clean. The agent perceives its location and the location's
    status."""

    def __init__(self, root, height=400, width=520):
        super().__init__()
        self.status = {loc_A: 'Dirty',
                       loc_B: 'Dirty', 
                       loc_C: 'Clean',
                       loc_D: 'Clean'}
        self.root = root
        self.height = height
        self.width = width
        self.canvas = None
        self.buttons = []
        self.create_canvas()
        self.create_buttons()

    def thing_classes(self):
        """The list of things which can be used in the environment."""
        return [Wall, Dirt, ReflexVacuumAgent, RandomVacuumAgent,
                TableDrivenVacuumAgent, ModelBasedVacuumAgent]

    def percept(self, agent):
        """Returns the agent's location, and the location status (Dirty/Clean)."""
        print('Perceived Location=', agent.location,' - Perceived Status=',  self.status)
        return (agent.location, self.status) 

    def execute_action(self, agent, action):
        """Change the location status (Dirty/Clean); track performance.
        Score 10 for each dirt cleaned; -1 for each move."""
        print('Executing action: ', action)
        
        if action == 'Right':
            if agent.location == loc_A:
                agent.location=loc_B
            elif agent.location == loc_C:
                agent.location = loc_D
            elif agent.location == loc_B:
                agent.location = loc_B
            else:
                agent.location = loc_D
            agent.performance -= 1

        elif action == 'Left':
            if agent.location == loc_D:
                agent.location = loc_C
            elif agent.location == loc_B:
                agent.location = loc_A
            elif agent.location == loc_A:
                agent.location = loc_A
            else:
                agent.location = loc_C
            agent.performance -= 1

        elif action == 'Up':
            if agent.location == loc_D:
                agent.location = loc_B
            elif agent.location == loc_C:
                agent.location = loc_A
            elif agent.location == loc_A:
                agent.location = loc_A
            else:
                agent.location = loc_B
            agent.performance -= 1

        elif action == 'Down':
            if agent.location == loc_A:
                agent.location = loc_C
            elif agent.location == loc_B:
                agent.location = loc_D
            elif agent.location == loc_C:
                agent.location = loc_C
            else:
                agent.location = loc_D
            agent.performance -= 1
        

        elif action == 'Suck':
            if self.status[agent.location] == 'Dirty':
                if agent.location == loc_A:
                    self.buttons[0].config(bg='white', activebackground='light grey')
                elif agent.location == loc_B:
                    self.buttons[1].config(bg='white', activebackground='light grey')
                elif agent.location == loc_C:
                    self.buttons[2].config(bg='white', activebackground='light grey')
                elif agent.location == loc_D:  # Aggiunta per loc_D
                    self.buttons[3].config(bg='white', activebackground='light grey')  # Cambia sfondo di loc_D
                agent.performance += 10
            self.status[agent.location] = 'Clean'


    def default_location(self, thing):
        """Agents start in either location at random."""
        return random.choice([loc_A, loc_B, loc_C, loc_D])

    ###################################################################################################
    #                        INTERFACE METHODS NOT USEFULL FOR THE AGENT/ENVIRONMENT
    ###################################################################################################
    def create_canvas(self):
        """Creates Canvas element in the GUI."""
        self.canvas = Canvas(
            self.root,
            width=self.width,
            height=self.height,
            background='powder blue')
        self.canvas.pack(side='bottom')

    def create_buttons(self):
        """Creates the buttons required in the GUI."""
        button_A = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_A.config(command=lambda btn=button_A: self.dirt_switch(btn))
        self.buttons.append(button_A)
        button_A_window = self.canvas.create_window(100, 100, anchor=N, window=button_A)

        button_B = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_B.config(command=lambda btn=button_B: self.dirt_switch(btn))
        self.buttons.append(button_B)
        button_B_window = self.canvas.create_window(400, 100, anchor=N, window=button_B)

        button_C = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_C.config(command=lambda btn=button_C: self.dirt_switch(btn))
        self.buttons.append(button_C)
        button_C_window = self.canvas.create_window(100, 300, anchor=N, window=button_C)

        button_D = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_D.config(command=lambda btn=button_D: self.dirt_switch(btn))
        self.buttons.append(button_D)
        button_D_window = self.canvas.create_window(400, 300, anchor=N, window=button_D)


    def dirt_switch(self, button):
        """Gives user the option to put dirt in any tile."""
        bg_color = button['bg']
        if bg_color == 'saddle brown':
            button.config(bg='white', activebackground='light grey')
        elif bg_color == 'white':
            button.config(bg='saddle brown', activebackground='light goldenrod')

    def read_env(self):
        """Reads the current state of the GUI."""
        for i, btn in enumerate(self.buttons):
            if i == 0:
                if btn['bg'] == 'white':
                    self.status[loc_A] = 'Clean'
                else:
                    self.status[loc_A] = 'Dirty'
            elif i == 1:
                if btn['bg'] == 'white':
                    self.status[loc_B] = 'Clean'
                else:
                    self.status[loc_B] = 'Dirty'
            elif i == 2:
                if btn['bg'] == 'white':
                    self.status[loc_C] = 'Clean'
                else:
                    self.status[loc_C] = 'Dirty'
            else:
                if btn['bg'] == 'white':
                    self.status[loc_D] = 'Clean'
                else:
                    self.status[loc_D] = 'Dirty'

    def update_env(self, agent):
        """Updates the GUI according to the agent's action."""
        self.read_env()
        # print(self.status)
        before_step = agent.location
        self.step()   #execute_actions
        print('Current Environment Status: ', self.status)
        print('Current Agent Location: ',agent.location)
        move_agent(self, agent, before_step)

###################################################################################################
#                                           THE AGENT
###################################################################################################

def ReflexVacuumAgent():
    """A reflex agent for the two-state vacuum environment. [Figure 2.8]
    >>> agent = ReflexVacuumAgent()
    >>> environment = TrivialVacuumEnvironment()
    >>> environment.add_thing(agent)
    >>> environment.run()
    >>> environment.status == {(1,0):'Clean' , (0,0) : 'Clean'}
    True
    """
    
    def program(percept):
        
        location, status = percept
        print('Perceived Status: ', percept)
        
        # TODO: fill this method and remove the next print!
        if status[location] == 'Dirty':
            return 'Suck'
        if status[location] == 'Clean':
            if location  == loc_A:
                if status[loc_B] == 'Dirty':
                    return 'Right'
                elif status[loc_C] == 'Dirty':
                    return 'Down'
                else:
                    return random.choice(['Right','Down'])
            elif location == loc_B:
                if status[loc_A] == 'Dirty':
                    return 'Left'
                elif status[loc_D] == 'Dirty':
                    return 'Down'
                else:
                    return random.choice(['Left','Down'])
            elif location == loc_C:
                if status[loc_D] == 'Dirty':
                    return 'Right'
                elif status[loc_A] == 'Dirty':
                    return 'Up'
                else:
                    return random.choice(['Right','Up'])
            elif location == loc_D:
                if status[loc_C] == 'Dirty':
                    return 'Left'
                elif status[loc_B] == 'Dirty':
                    return 'Up'
                else:
                    return random.choice(['Left','Up'])

        return "Stay"

    return Agent(program)

###################################################################################################
#                            INTERFACE METHODS TO CREATE AND MOVE THE AGENT
###################################################################################################

def create_agent(env, agent):
    """Creates the agent in the GUI and is kept independent of the environment."""
    env.add_thing(agent)

    if agent.location == loc_A:
        env.text = env.canvas.create_text(100, 80, font="Helvetica 10 bold italic", text="Agent")  
        env.agent_logo = env.canvas.create_image(100, 70)  

    elif agent.location == loc_B:
        env.text = env.canvas.create_text(400, 80, font="Helvetica 10 bold italic", text="Agent")
        env.agent_logo = env.canvas.create_image(400, 70)

    elif agent.location == loc_C:
        env.text = env.canvas.create_text(100, 280, font="Helvetica 10 bold italic", text="Agent")
        env.agent_logo = env.canvas.create_image(100, 270)

    elif agent.location == loc_D:
        env.text = env.canvas.create_text(400, 280, font="Helvetica 10 bold italic", text="Agent")
        env.agent_logo = env.canvas.create_image(400, 270)




def move_agent(env, agent, before_step):
    """Moves the agent in the GUI when 'next' button is pressed."""
    if agent.location != before_step:
        # Rimuove il testo e l'immagine precedenti
        env.canvas.delete(env.text)
        env.canvas.delete(env.agent_logo)

        # Posiziona il testo e l'immagine nella nuova posizione
        if agent.location == loc_A:
            env.text = env.canvas.create_text(100, 80, font="Helvetica 10 bold italic", text="Agent")
            env.agent_logo = env.canvas.create_image(100, 70,)
        elif agent.location == loc_B:
            env.text = env.canvas.create_text(400, 80, font="Helvetica 10 bold italic", text="Agent")
            env.agent_logo = env.canvas.create_image(400, 70)
        elif agent.location == loc_C:
            env.text = env.canvas.create_text(100, 280, font="Helvetica 10 bold italic", text="Agent")
            env.agent_logo = env.canvas.create_image(100, 270)
        elif agent.location == loc_D:
            env.text = env.canvas.create_text(400, 280, font="Helvetica 10 bold italic", text="Agent")
            env.agent_logo = env.canvas.create_image(400, 270)


# TODO: Add more agents to the environment.
# TODO: Expand the environment to XYEnvironment.

###################################################################################################
#                                           MAIN LOOP
###################################################################################################
def main():
    """The main function of the program."""
    root = Tk()
    root.title("Vacuum Environment")
    root.geometry("500x500")
    root.resizable(0, 0)
    frame = Frame(root, bg='black')
    #MyImage = PhotoImage(file="AgentVacuum.png")
    next_button = Button(frame, text='Next', height=2, width=6, padx=2, pady=2)
    next_button.pack(side='left')
    frame.pack(side='bottom')
    
    env = Gui(root)
    agent = ReflexVacuumAgent()
    create_agent(env, agent)
    
    next_button.config(command=lambda: env.update_env(agent))
    
    root.mainloop()


if __name__ == "__main__":
    main()

**Esercizion 4**
Modify the `Execute action` method of the Environment in order to implement a probabilistic failure of the action chosen by the Agent. In that case, the action should fail and nothing happens!

**Soluzine :**
Nella funzione `execute_action`, ho implementato una probabilità di fallimento pari a 1/5 per l'esecuzione dell'azione dell'agente. Questo significa che, nel 20% dei casi, l'agente non riuscirà a eseguire l'azione richiesta

In [ ]:
from tkinter import *
import random
from agents import *

loc_A, loc_B, loc_C, loc_D= (0, 0), (1, 0), (0,1), (1,1) 

class Gui(Environment):

    """This GUI environment has three locations, A, B and C. Each can be Dirty
    or Clean. The agent perceives its location and the location's
    status."""

    def __init__(self, root, height=400, width=520):
        super().__init__()
        self.status = {loc_A: 'Dirty',
                       loc_B: 'Dirty', 
                       loc_C: 'Clean',
                       loc_D: 'Clean'}
        self.root = root
        self.height = height
        self.width = width
        self.canvas = None
        self.buttons = []
        self.create_canvas()
        self.create_buttons()

    def thing_classes(self):
        """The list of things which can be used in the environment."""
        return [Wall, Dirt, ReflexVacuumAgent, RandomVacuumAgent,
                TableDrivenVacuumAgent, ModelBasedVacuumAgent]

    def percept(self, agent):
        """Returns the agent's location, and the location status (Dirty/Clean)."""
        print('Perceived Location=', agent.location,' - Perceived Status=',  self.status)
        return (agent.location, self.status) 

    def execute_action(self, agent, action):
        """Change the location status (Dirty/Clean); track performance.
        Score 10 for each dirt cleaned; -1 for each move."""
        print('Executing action: ', action)

        if(random.choice([True,False,False,False,False])):
            agent.performance -=1
            return 
        
        if action == 'Right':
            if agent.location == loc_A:
                agent.location=loc_B
            elif agent.location == loc_C:
                agent.location = loc_D
            elif agent.location == loc_B:
                agent.location = loc_B
            else:
                agent.location = loc_D
            agent.performance -= 1

        elif action == 'Left':
            if agent.location == loc_D:
                agent.location = loc_C
            elif agent.location == loc_B:
                agent.location = loc_A
            elif agent.location == loc_A:
                agent.location = loc_A
            else:
                agent.location = loc_C
            agent.performance -= 1

        elif action == 'Up':
            if agent.location == loc_D:
                agent.location = loc_B
            elif agent.location == loc_C:
                agent.location = loc_A
            elif agent.location == loc_A:
                agent.location = loc_A
            else:
                agent.location = loc_B
            agent.performance -= 1

        elif action == 'Down':
            if agent.location == loc_A:
                agent.location = loc_C
            elif agent.location == loc_B:
                agent.location = loc_D
            elif agent.location == loc_C:
                agent.location = loc_C
            else:
                agent.location = loc_D
            agent.performance -= 1
        

        elif action == 'Suck':
            if self.status[agent.location] == 'Dirty':
                if agent.location == loc_A:
                    self.buttons[0].config(bg='white', activebackground='light grey')
                elif agent.location == loc_B:
                    self.buttons[1].config(bg='white', activebackground='light grey')
                elif agent.location == loc_C:
                    self.buttons[2].config(bg='white', activebackground='light grey')
                elif agent.location == loc_D:  # Aggiunta per loc_D
                    self.buttons[3].config(bg='white', activebackground='light grey')  # Cambia sfondo di loc_D
                agent.performance += 10
            self.status[agent.location] = 'Clean'


    def default_location(self, thing):
        """Agents start in either location at random."""
        return random.choice([loc_A, loc_B, loc_C, loc_D])

    ###################################################################################################
    #                        INTERFACE METHODS NOT USEFULL FOR THE AGENT/ENVIRONMENT
    ###################################################################################################
    def create_canvas(self):
        """Creates Canvas element in the GUI."""
        self.canvas = Canvas(
            self.root,
            width=self.width,
            height=self.height,
            background='powder blue')
        self.canvas.pack(side='bottom')

    def create_buttons(self):
        """Creates the buttons required in the GUI."""
        button_A = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_A.config(command=lambda btn=button_A: self.dirt_switch(btn))
        self.buttons.append(button_A)
        button_A_window = self.canvas.create_window(100, 100, anchor=N, window=button_A)

        button_B = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_B.config(command=lambda btn=button_B: self.dirt_switch(btn))
        self.buttons.append(button_B)
        button_B_window = self.canvas.create_window(400, 100, anchor=N, window=button_B)

        button_C = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_C.config(command=lambda btn=button_C: self.dirt_switch(btn))
        self.buttons.append(button_C)
        button_C_window = self.canvas.create_window(100, 300, anchor=N, window=button_C)

        button_D = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_D.config(command=lambda btn=button_D: self.dirt_switch(btn))
        self.buttons.append(button_D)
        button_D_window = self.canvas.create_window(400, 300, anchor=N, window=button_D)


    def dirt_switch(self, button):
        """Gives user the option to put dirt in any tile."""
        bg_color = button['bg']
        if bg_color == 'saddle brown':
            button.config(bg='white', activebackground='light grey')
        elif bg_color == 'white':
            button.config(bg='saddle brown', activebackground='light goldenrod')

    def read_env(self):
        """Reads the current state of the GUI."""
        for i, btn in enumerate(self.buttons):
            if i == 0:
                if btn['bg'] == 'white':
                    self.status[loc_A] = 'Clean'
                else:
                    self.status[loc_A] = 'Dirty'
            elif i == 1:
                if btn['bg'] == 'white':
                    self.status[loc_B] = 'Clean'
                else:
                    self.status[loc_B] = 'Dirty'
            elif i == 2:
                if btn['bg'] == 'white':
                    self.status[loc_C] = 'Clean'
                else:
                    self.status[loc_C] = 'Dirty'
            else:
                if btn['bg'] == 'white':
                    self.status[loc_D] = 'Clean'
                else:
                    self.status[loc_D] = 'Dirty'

    def update_env(self, agent):
        """Updates the GUI according to the agent's action."""
        self.read_env()
        # print(self.status)
        before_step = agent.location
        self.step()   #execute_actions
        print('Current Environment Status: ', self.status)
        print('Current Agent Location: ',agent.location)
        move_agent(self, agent, before_step)

###################################################################################################
#                                           THE AGENT
###################################################################################################

def ReflexVacuumAgent():
    """A reflex agent for the two-state vacuum environment. [Figure 2.8]
    >>> agent = ReflexVacuumAgent()
    >>> environment = TrivialVacuumEnvironment()
    >>> environment.add_thing(agent)
    >>> environment.run()
    >>> environment.status == {(1,0):'Clean' , (0,0) : 'Clean'}
    True
    """
    
    def program(percept):
        
        location, status = percept
        print('Perceived Status: ', percept)
        
        # TODO: fill this method and remove the next print!
        if status[location] == 'Dirty':
            return 'Suck'
        if status[location] == 'Clean':
            if location  == loc_A:
                if status[loc_B] == 'Dirty':
                    return 'Right'
                elif status[loc_C] == 'Dirty':
                    return 'Down'
                else:
                    return random.choice(['Right','Down'])
            elif location == loc_B:
                if status[loc_A] == 'Dirty':
                    return 'Left'
                elif status[loc_D] == 'Dirty':
                    return 'Down'
                else:
                    return random.choice(['Left','Down'])
            elif location == loc_C:
                if status[loc_D] == 'Dirty':
                    return 'Right'
                elif status[loc_A] == 'Dirty':
                    return 'Up'
                else:
                    return random.choice(['Right','Up'])
            elif location == loc_D:
                if status[loc_C] == 'Dirty':
                    return 'Left'
                elif status[loc_B] == 'Dirty':
                    return 'Up'
                else:
                    return random.choice(['Left','Up'])

        return "Stay"

    return Agent(program)

###################################################################################################
#                            INTERFACE METHODS TO CREATE AND MOVE THE AGENT
###################################################################################################

def create_agent(env, agent):
    """Creates the agent in the GUI and is kept independent of the environment."""
    env.add_thing(agent)

    if agent.location == loc_A:
        env.text = env.canvas.create_text(100, 80, font="Helvetica 10 bold italic", text="Agent")  
        env.agent_logo = env.canvas.create_image(100, 70)  

    elif agent.location == loc_B:
        env.text = env.canvas.create_text(400, 80, font="Helvetica 10 bold italic", text="Agent")
        env.agent_logo = env.canvas.create_image(400, 70)

    elif agent.location == loc_C:
        env.text = env.canvas.create_text(100, 280, font="Helvetica 10 bold italic", text="Agent")
        env.agent_logo = env.canvas.create_image(100, 270)

    elif agent.location == loc_D:
        env.text = env.canvas.create_text(400, 280, font="Helvetica 10 bold italic", text="Agent")
        env.agent_logo = env.canvas.create_image(400, 270)




def move_agent(env, agent, before_step):
    """Moves the agent in the GUI when 'next' button is pressed."""
    if agent.location != before_step:
        # Rimuove il testo e l'immagine precedenti
        env.canvas.delete(env.text)
        env.canvas.delete(env.agent_logo)

        # Posiziona il testo e l'immagine nella nuova posizione
        if agent.location == loc_A:
            env.text = env.canvas.create_text(100, 80, font="Helvetica 10 bold italic", text="Agent")
            env.agent_logo = env.canvas.create_image(100, 70)
        elif agent.location == loc_B:
            env.text = env.canvas.create_text(400, 80, font="Helvetica 10 bold italic", text="Agent")
            env.agent_logo = env.canvas.create_image(400, 70)
        elif agent.location == loc_C:
            env.text = env.canvas.create_text(100, 280, font="Helvetica 10 bold italic", text="Agent")
            env.agent_logo = env.canvas.create_image(100, 270)
        elif agent.location == loc_D:
            env.text = env.canvas.create_text(400, 280, font="Helvetica 10 bold italic", text="Agent")
            env.agent_logo = env.canvas.create_image(400, 270)


# TODO: Add more agents to the environment.
# TODO: Expand the environment to XYEnvironment.

###################################################################################################
#                                           MAIN LOOP
###################################################################################################
def main():
    """The main function of the program."""
    root = Tk()
    root.title("Vacuum Environment")
    root.geometry("500x500")
    root.resizable(0, 0)
    frame = Frame(root, bg='black')
    MyImage = PhotoImage(file="AgentVacuum.png")
    next_button = Button(frame, text='Next', height=2, width=6, padx=2, pady=2)
    next_button.pack(side='left')
    frame.pack(side='bottom')
    
    env = Gui(root)
    agent = ReflexVacuumAgent()
    create_agent(env, agent)
    
    next_button.config(command=lambda: env.update_env(agent))
    
    root.mainloop()


if __name__ == "__main__":
    main()